<a href="https://colab.research.google.com/github/anita-maxwynn/Crypto-shit/blob/main/Eliptic_Curve.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
import secrets
import os

In [2]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __eq__(self, other):
        if other is None:
            return False
        return (self.x == other.x) and (self.y == other.y)

    def __repr__(self):
        return f"Point({self.x}, {self.y})"

    def __str__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"

In [3]:
def inverse_mod_p(a, p):
    """Return modular inverse of a mod p. Assumes p is prime and a % p != 0."""
    return pow(a, -1, p)

In [4]:
class Elipic_Curve:
    def __init__(self, a, b, p,salt):
        self.a = a
        self.b = b
        self.p = p
        self.salt=salt

    def __str__(self):
        return f"({self.a},{self.b},{self.p},{self.salt})"

    def is_on_curve(self, P):
        """Return True if point P is on the curve or P is None (point at infinity)."""
        if P is None:
            return True
        return (P.y * P.y - (P.x * P.x * P.x) - self.a * P.x - self.b) % self.p == 0

    def add(self, P, Q):
        """Add two points P and Q on the curve (handle point at infinity)."""
        # identity cases
        if P is None:
            return Q
        if Q is None:
            return P

        # P + (-P) = O
        if (P.x % self.p) == (Q.x % self.p) and ((P.y + Q.y) % self.p) == 0:
            return None

        if P == Q:
            # point doubling
            if (P.y % self.p) == 0:
                return None  # tangent is vertical -> point at infinity
            num = (3 * (P.x * P.x) + self.a) % self.p
            den = (2 * P.y) % self.p
            l = (num * inverse_mod_p(den, self.p)) % self.p
        else:
            # distinct points
            if (P.x - Q.x) % self.p == 0:
                return None
            num = (Q.y - P.y) % self.p
            den = (Q.x - P.x) % self.p
            l = (num * inverse_mod_p(den, self.p)) % self.p

        x_r = (l * l - P.x - Q.x) % self.p
        y_r = (l * (P.x - x_r) - P.y) % self.p
        return Point(x_r, y_r)

    def double(self, P):
        """Return 2P."""
        return self.add(P, P)

    def neg_point(self, P):
        """Return -P."""
        if P is None:
            return None
        return Point(P.x, (-P.y) % self.p)

    def scaler_multiply(self, k, P):
        """
        Scalar multiplication k * P using double-and-add.
        Accepts negative k (uses negation).
        FIXED: handle k == 0, negative k, and avoid using self.p in k modulus.
        """
        if P is None or k == 0:
            return None
        if k < 0:
            return self.scaler_multiply(-k, self.neg_point(P))

        result = None
        addend = P

        while k:
            if k & 1:
                result = self.add(result, addend)
            addend = self.double(addend)
            k >>= 1
        return result

    def legendre_symbol(self, a):
        """Compute the Legendre symbol (a|p) where p = self.p. Returns 0,1,-1."""
        a = a % self.p
        if a == 0:
            return 0
        ls = pow(a, (self.p - 1) // 2, self.p)
        return -1 if ls == self.p - 1 else ls

    def sqr_root_mod_p(self, a):
        """
        Tonelli-Shanks: return None if no sqrt, otherwise return [r, p-r].
        Uses self.p (prime).
        """
        a = a % self.p
        if a == 0:
            return [0]
        if self.p == 2:
            return [a]
        if self.legendre_symbol(a) != 1:
            return None

        # Simple case p % 4 == 3
        if self.p % 4 == 3:
            r = pow(a, (self.p + 1) // 4, self.p)
            return [r, (-r) % self.p]

        # Factor p-1 as Q * 2^S with Q odd
        Q = self.p - 1
        S = 0
        while Q % 2 == 0:
            Q //= 2
            S += 1

        # find a quadratic non-residue z
        z = 2
        while self.legendre_symbol(z) != -1:
            z += 1

        c = pow(z, Q, self.p)
        R = pow(a, (Q + 1) // 2, self.p)
        t = pow(a, Q, self.p)
        M = S

        while t % self.p != 1:
            # find smallest i, 0 < i < M, such that t^(2^i) == 1
            i = 1
            t2 = (t * t) % self.p
            while t2 != 1:
                t2 = (t2 * t2) % self.p
                i += 1
                if i == M:
                    raise ValueError("sqr_root_mod_p failed to find i")
            b = pow(c, 1 << (M - i - 1), self.p)
            R = (R * b) % self.p
            c = (b * b) % self.p
            t = (t * c) % self.p
            M = i

        return [R, (-R) % self.p]

    # ---------------- helpers to (de)serialize points and ints ----------------
    def int_to_bytes_fixed(self, v, length):
        return v.to_bytes(length, "big")

    def bytes_to_int(self, b):
        return int.from_bytes(b, "big")

    def point_to_bytes(self, P):
        if P is None:
            raise ValueError("cannot serialize point at infinity")
        field_len = (self.p.bit_length() + 7) // 8
        bx = self.int_to_bytes_fixed(P.x, field_len)
        by = self.int_to_bytes_fixed(P.y, field_len)
        return bx + by

    def bytes_to_point(self, data):
        field_len = (self.p.bit_length() + 7) // 8
        if len(data) != 2 * field_len:
            raise ValueError("bad point encoding length")
        bx = data[:field_len]
        by = data[field_len:]
        x = int.from_bytes(bx, "big")
        y = int.from_bytes(by, "big")
        P = Point(x, y)
        if not self.is_on_curve(P):
            raise ValueError("point not on curve")
        return P

    # ---------------- ECIES-like encrypt/decrypt (unchanged) ----------------
    def ecies_encrypt(self, recipient_Q, plaintext, info=b"ECIES-v1"):
        eph_priv = secrets.randbelow(n - 1) + 1
        eph_pub = self.scaler_multiply(eph_priv, G)
        S = self.scaler_multiply(eph_priv, recipient_Q)
        if S is None:
            raise ValueError("invalid shared secret")

        field_len = (self.p.bit_length() + 7) // 8
        shared_bytes = self.int_to_bytes_fixed(S.x, field_len)

        hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=info)
        sym_key = hkdf.derive(shared_bytes)

        aesgcm = AESGCM(sym_key)
        nonce = os.urandom(12)
        ciphertext = aesgcm.encrypt(nonce, plaintext, associated_data=None)
        return {"ephemeral_pub": self.point_to_bytes(eph_pub), "nonce": nonce, "ciphertext": ciphertext}

    def ecies_decrypt(self, recipient_priv, bundle, info=b"ECIES-v1"):
        eph_pub = self.bytes_to_point(bundle["ephemeral_pub"])
        S = self.scaler_multiply(recipient_priv, eph_pub)
        if S is None:
            raise ValueError("invalid shared secret on decrypt")

        field_len = (self.p.bit_length() + 7) // 8
        shared_bytes = self.int_to_bytes_fixed(S.x, field_len)
        hkdf = HKDF(algorithm=hashes.SHA256(), length=32, salt=None, info=info)
        sym_key = hkdf.derive(shared_bytes)

        aesgcm = AESGCM(sym_key)
        plaintext = aesgcm.decrypt(bundle["nonce"], bundle["ciphertext"], associated_data=None)
        return plaintext


In [5]:
import hashlib

class Elipic_Curve(Elipic_Curve):  # extend your existing class
    def ecdsa_sign(self, msg, priv):
        e = int.from_bytes(hashlib.sha256(msg).digest(), "big")
        while True:
            k = secrets.randbelow(n - 1) + 1
            R = self.scaler_multiply(k, G)
            r = R.x % n
            if r == 0:
                continue
            k_inv = pow(k, -1, n)
            s = (k_inv * (e + priv * r)) % n
            if s != 0:
                break
        return (r, s)

    def ecdsa_verify(self, msg, signature, pub):
        r, s = signature
        if not (1 <= r < n and 1 <= s < n):
            return False
        e = int.from_bytes(hashlib.sha256(msg).digest(), "big")
        w = pow(s, -1, n)
        u1 = (e * w) % n
        u2 = (r * w) % n
        X = self.add(self.scaler_multiply(u1, G),
                     self.scaler_multiply(u2, pub))
        if X is None:
            return False
        return (X.x % n) == r


In [6]:
# secp256k1 parameters
p = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEFFFFFC2F
a = 0
b = 7
Gx = 55066263022277343669578718895168534326250603453777594175500187360389116729240
Gy = 32670510020758816978083085130507043184471273380659243275938904335757337482424
n = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFEBAAEDCE6AF48A03BBFD25E8CD0364141  # order of G
salt = os.urandom(16)

In [7]:
curve = Elipic_Curve(a, b, p,salt)
print(curve)
G = Point(Gx, Gy)
print(G)

(0,7,115792089237316195423570985008687907853269984665640564039457584007908834671663,b'IO\x8d\xe8~\xb5\xfe!R<\x96\t~\xf4cg')
(55066263022277343669578718895168534326250603453777594175500187360389116729240,32670510020758816978083085130507043184471273380659243275938904335757337482424)


In [8]:
import secrets
# Reciever private key
d = secrets.randbelow(n - 1) + 1
print("Private key d:", d)


Private key d: 11455676176183148386573211445734049647309862430819547841733778849985700436473


In [9]:
# Reciever public key
Q = curve.scaler_multiply(d, G)
print("Public key Q:", Q)

Public key Q: (28736818726222121731898590424111381382250720398196963430207771288399512162010,67487659680413576596411187466153609266769265001522230055773190066810432943880)


In [10]:
## Sender side
pri = secrets.randbelow(n-1)+1
pub = curve.scaler_multiply(pri,G)

In [11]:
# encrypt/decrypt
plaintext = b"Hello, ECIES-like hybrid!"
bundle = curve.ecies_encrypt(Q, plaintext)
sig = curve.ecdsa_sign(plaintext, pri)
print("Bundle:", bundle)
print("Signature:", sig)

recovered = curve.ecies_decrypt(d, bundle)
valid = curve.ecdsa_verify(recovered, sig, pub)
print(valid)
print("Recovered:", recovered)

Bundle: {'ephemeral_pub': b'\xce\xf3\xab\xb6\xee{\x94\xe0\xc9iV]\x04\xd5\xbe\xcb\x07sk\xefgO$3\xad5\xd2l:\x90\xd8&\xd26\xd68\x18\x03\x14y!.\xbb\xb7h\x9ab\xf8\xbcw;~\xeb\xb8\xf3u\xf8\xe4J\xb2\xbf\x0bN&', 'nonce': b'\x8bG\xe1\xfb\xd6/\xcf\xc7\xf5/\x98\xb3', 'ciphertext': b';\xf2z\xb5\xe4\x16c\x98\xe7E4\xc4\x0e5%F\xf50\x0bD\xa8\rf\x95C/\xf2N]\xe0\xf0z\x8d$\xde&GH5\xdfe'}
Signature: (72955171310556879319647668068736474565108039004342392108752807513820493697114, 25068423109364365928624575830530137530295144174042010181581117872070806626497)
True
Recovered: b'Hello, ECIES-like hybrid!'
